<a href="https://colab.research.google.com/github/Khushipant/ai-agent/blob/main/BreakoutAI_AIAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install langchain langchain-groq tabulate openpyxl streamlit langchain_community pyngrok flask gspread oauth2client

In [3]:
# Import required libraries
import streamlit as st
import pandas as pd
import os
import gspread
from google.colab import auth
from google.auth import default
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq
from flask import Flask, render_template, request, jsonify
from pyngrok import ngrok
from langchain_groq import ChatGroq

In [4]:
# Set up the LLM chain
llm = ChatGroq(temperature=0.8, groq_api_key='xyz', model_name="llama3-70b-8192") #replace xyz with your groq_api

In [5]:
prompt_template = """
I have the following CSV data with the columns: "{columns}".
Data is in a dataframe called 'df' already.
Don't give any description or explanation or any English sentence,
just write relevant Python code and store output in a variable called result.
Please generate a Python script using this 'df' as input dataframe and pandas to answer this question: "{question}".
Do not write any Python script which alters the dataframe 'df'.
Write only read-only Python script.
"""

In [6]:
template = PromptTemplate(
    input_variables=["columns", "question"],
    template=prompt_template,
)

llm_chain = LLMChain(prompt=template, llm=llm)

<ipython-input-6-7c34c4a85100>:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=template, llm=llm)


In [7]:
# Google Sheets authentication
def authenticate_google_sheets():
    auth.authenticate_user()
    creds, _ = default()
    return gspread.authorize(creds)

In [8]:
# File handling logic
def handle_uploaded_file(uploaded_file):
    df = pd.DataFrame()

    if uploaded_file.endswith('.csv'):
        df = pd.read_csv(uploaded_file)
    elif uploaded_file.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(uploaded_file)
    elif uploaded_file.startswith("https://docs.google.com/spreadsheets/"):
        try:
            sheet_key = uploaded_file.split('/')[-2]
            gc = authenticate_google_sheets()
            worksheet = gc.open_by_key(sheet_key).sheet1
            rows = worksheet.get_all_values()
            df = pd.DataFrame(rows)
        except Exception as e:
            print(f"Error reading Google Sheet: {e}")

    return df

In [24]:
# Flask app setup
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        uploaded_file = None
        error = None
        df = pd.DataFrame()

        # Handle uploaded file
        if 'file' in request.files and request.files['file'].filename:
            file = request.files['file']
            uploaded_file = file.filename
            file.save(uploaded_file)  # Save the file temporarily
            try:
                df = handle_uploaded_file(uploaded_file)
            except Exception as e:
                error = f"Error processing file: {e}"

        # Handle Google Sheet URL
        elif 'google_sheet_url' in request.form:
            uploaded_file = request.form['google_sheet_url']
            if uploaded_file.startswith('https://docs.google.com/spreadsheets/'):
                try:
                    df = handle_uploaded_file(uploaded_file)
                except Exception as e:
                    error = f"Error accessing Google Sheet: {e}"
            else:
                error = "Invalid Google Sheet URL."

        # Retrieve question
        question = request.form.get('question', '')

        # Process the question and dataframe
        answer = None
        if not df.empty and not error:
            try:
                # Generate Python code to answer the question
                python_script = llm_chain.invoke({
                    "columns": ", ".join(df.columns),
                    "question": question
                })

                # Display the generated Python code
                print("### Generated Python Code:")
                print(python_script['text'].strip('`'))

                # Execute the generated Python code
                exec_globals = {"df": df, "pd": pd}
                exec_locals = {}
                python_script['text'] = python_script['text'].strip('`')
                exec(python_script['text'], exec_globals, exec_locals)

                # If a result variable is present, store and print it
                answer = exec_globals.get('result', exec_locals.get('result'))
            except Exception as e:
                error = f"Error processing the question: {e}"

        # Return JSON response
        if answer:
            return jsonify(answer)
        else:
            return jsonify({'error': error or "Unable to process input."})

    return render_template('index.html')

In [25]:
# Streamlit app setup
def create_html_form():
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Data Processing</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f9;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }

            .form-container {
                background: #ffffff;
                padding: 20px 30px;
                border-radius: 8px;
                box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
                width: 100%;
                max-width: 400px;
                margin-bottom: 20px;
            }

            h1 {
                text-align: center;
                color: #333333;
                margin-bottom: 20px;
            }

            form {
                display: flex;
                flex-direction: column;
            }

            input[type='file'],
            input[type='text'],
            input[type='submit'] {
                margin-bottom: 15px;
                padding: 10px;
                font-size: 16px;
                border: 1px solid #cccccc;
                border-radius: 4px;
                box-sizing: border-box;
            }

            input[type='submit'] {
                background-color: #007BFF;
                color: #ffffff;
                border: none;
                cursor: pointer;
                transition: background-color 0.3s ease;
            }

            input[type='submit']:hover {
                background-color: #0056b3;
            }

            .separator {
                text-align: center;
                margin: 10px 0;
                color: #666666;
            }

            .note {
                font-size: 14px;
                color: #888888;
                text-align: center;
            }

            .response {
                margin-top: 20px;
                padding: 15px;
                background-color: #eaf7ea;
                border: 1px solid #c1e2c1;
                border-radius: 8px;
                color: #2d662d;
                font-size: 16px;
            }
        </style>
    </head>
    <body>
        <div class="form-container">
            <h1>Data Processing</h1>
            <form method="POST" enctype="multipart/form-data">
                <label for="file">Upload File:</label>
                <input type="file" name="file" id="file"><br>
                <div class="separator">OR</div>
                <label for="google_sheet_url">Google Sheet URL:</label>
                <input type="text" name="google_sheet_url" id="google_sheet_url" placeholder="Enter Google Sheet URL"><br>
                <label for="question">Question:</label>
                <input type="text" name="question" id="question" placeholder="Enter your question"><br>
                <input type="submit" value="Submit">
            </form>
            <p class="note">Supported file formats: .csv, .xls, .xlsx</p>
        </div>

        {% if answer %}
        <div class="response">
            <strong>Generated Result:</strong>
            <p>{{ answer }}</p>
        </div>
        {% endif %}
    </body>
    </html>
    """

    with open("templates/index.html", "w") as file:
        file.write(html_content)

In [26]:
!pip install pyngrok
from pyngrok import ngrok
!ngrok config add-authtoken xyz # replace xyz with your ngrock authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [27]:
from pyngrok import ngrok

# Expose the Flask app via ngrok
public_url = ngrok.connect(8001)
print(' * Tunnel URL:', public_url)


 * Tunnel URL: NgrokTunnel: "https://b51f-34-138-25-244.ngrok-free.app" -> "http://localhost:8001"
